In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

def get_article_content(link):
    try:
        res = requests.get(link, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')

        # Menemukan div dengan class yang sesuai
        div_content = soup.find('div', class_='mt-3 o_wblog_post_content_field js_tweet o_wblog_read_text')

        # Mengambil seluruh teks yang ada di dalam div_content dan menggabungkan berdasarkan <br>
        content = ''
        for br_tag in div_content.find_all('br'):
            # Dapatkan teks sebelum <br> dan tambahkan ke content
            text_before_br = br_tag.find_previous_sibling(text=True)
            if text_before_br:
                content += text_before_br.strip() + ' '  # Menambahkan spasi antar bagian

        # Untuk memastikan kita mengambil teks setelah <br> terakhir
        final_text = div_content.get_text(strip=True).replace('\n', ' ').replace('\r', '')
        return final_text.strip()
    except Exception as e:
        print(f"Error artikel: {link} | {e}")
        return ''

# url_first = 'https://tribratanews.polri.go.id/blog/none-22?search=timnas+indonesia'
# res_first = requests.get(url_first, headers=headers)
# soup_first = BeautifulSoup(res_first.text, 'lxml')

# page_numbers = []
# for a in soup_first.find_all('a'):
#     try:
#         num = int(a.text.strip())
#         page_numbers.append(num)
#     except:
#         continue

# last_page = max(page_numbers)
# print(f"🔎 Total halaman ditemukan {last_page}")

# with open('tribata.csv', 'w', newline='', encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerow(['Title', 'Link', 'Time', 'Content'])

#     for page in range(1, last_page + 1):
#         print(f"\n📄 Scraping halaman {page}...")
#         url = f'https://tribratanews.polri.go.id/blog/none-22/page/{page}?search=timnas+indonesia'
#         try:
#             res = requests.get(url, headers=headers)
#             soup = BeautifulSoup(res.text, 'lxml')
#             articles = soup.find_all('article')

#             for art in articles:
#                 try:
#                     a_tag = art.find('a', class_='text-reset text-decoration-none')
#                     title = a_tag.text.strip()
#                     link = a_tag['href']
#                     time_info = art.find('span', class_='text-muted pl-2 pr-1').text.strip()
#                     content = get_article_content(link)
#                     writer.writerow([title, link, time_info, content])
#                     print(f"✅ {title}")
#                     time.sleep(1)
#                 except Exception as e:
#                     print(f"Error parsing artikel di halaman {page}: {e}")
#                     continue

#             time.sleep(1)
#         except Exception as e:
#             print(f"Gagal akses page {page}: {e}")
#             continue

import requests
from bs4 import BeautifulSoup
import csv
import time
from urllib.parse import urljoin  # We will use this to combine the base URL with relative URLs

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

# Base URL for the site
base_url = "https://tribratanews.polri.go.id"

# Function to fetch article content
def get_article_content(link):
    try:
        # Ensure the link is a full URL (combine the base URL with the relative link if necessary)
        full_url = urljoin(base_url, link)

        res = requests.get(full_url, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')

        # Find the content within the div that contains the main content
        div_content = soup.find('div', class_='mt-3 o_wblog_post_content_field js_tweet o_wblog_read_text')

        if div_content:
            return div_content.get_text(strip=True).replace('\n', ' ').replace('\r', '')
        else:
            return 'No content found'
    except Exception as e:
        print(f"Error getting content from {link}: {e}")
        return 'Error retrieving content'

# First page scraping to get the total number of pages
url_first = 'https://tribratanews.polri.go.id/blog/none-22?search=timnas+indonesia'
res_first = requests.get(url_first, headers=headers)
soup_first = BeautifulSoup(res_first.text, 'lxml')

page_numbers = []
for a in soup_first.find_all('a'):
    try:
        num = int(a.text.strip())
        page_numbers.append(num)
    except ValueError:
        continue  # Skip non-numeric links

last_page = max(page_numbers) if page_numbers else 1  # Default to 1 if no page numbers are found
print(f"🔎 Total halaman ditemukan {last_page}")

# Write the scraped data to a CSV file
with open('tribata.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Link', 'Time', 'Content'])

    # Scrape each page
    for page in range(1, last_page + 1):
        print(f"\n📄 Scraping halaman {page}...")
        url = f'https://tribratanews.polri.go.id/blog/none-22/page/{page}?search=timnas+indonesia'
        try:
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            articles = soup.find_all('article')

            for art in articles:
                try:
                    a_tag = art.find('a', class_='text-reset text-decoration-none')
                    title = a_tag.text.strip()
                    link = a_tag['href']
                    time_info = art.find('span', class_='text-muted pl-2 pr-1').text.strip()
                    content = get_article_content(link)  # Get the article content
                    writer.writerow([title, link, time_info, content])
                    print(f"✅ {title}")
                    time.sleep(1)  # Adding a delay between articles
                except Exception as e:
                    print(f"Error parsing article on page {page}: {e}")
                    continue

            time.sleep(1)  # Delay between page requests
        except Exception as e:
            print(f"Failed to access page {page}: {e}")
            continue



🔎 Total halaman ditemukan 3

📄 Scraping halaman 1...
✅ [HOAKS] PSSI Tolak Ajakan FIFA untuk ke Piala Dunia Menggantikan Iran
✅ [HOAKS] Shin Tae-Yong Balik ke PSSI
✅ [HOAKS] Patrick Kluivert Undur Diri dari Posisi Pelatih Timnas Indonesia
✅ [HOAKS] Dibatalkannya Kemenangan Timnas Korut atas Indonesia di Piala Asia U-17
✅ [HOAKS] Maarten Paes Dikeluarkan dari Timnas
✅ [HOAKS] STY Kembali Latih Timnas Indonesia
✅ [HOAKS] Ronaldo Tiba di Kota Kupang dengan Pesawat
✅ [HOAKS] Bintang Muda Arda Guler Dinaturalisasi
✅ [HOAKS] Bung Towel Dikeroyok 6 Orang Tak Dikenal
✅ [HOAKS] Presiden Prabowo Minta PSSI Tarik Shin Tae-yong Kembali
✅ [HOAKS] Video Louis Van Gaal Tertarik Melatih Timnas Insonesia
✅ [HOAKS] Timnas Indonesia Resmi Keluar dari AFF
✅ [HOAKS] Pemain Timnas Arab Saudi Sebut Stadion GBK seperti Neraka
✅ [HOAKS] Pemain Timnas Jepang Resmi Peluk Indonesia
✅ [HOAKS] Timnas Indonesia U15 Jadi Juara Pildun 2024

📄 Scraping halaman 2...
✅ [HOAKS] Wasit Bahrain Dipidana Delapan Tahun Penjara


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

def get_article_content(link):
    try:
        res = requests.get(link, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')
        div_content = soup.find('div', class_='mt-3 o_wblog_post_content_field js_tweet  o_wblog_read_text')
        paragraphs = div_content.find_all('p')
        content = ' '.join([p.get_text(strip=True) for p in paragraphs])
        return content
    except Exception as e:
        print(f"Error artikel: {link} | {e}")
        return ''

url_first = 'https://tribratanews.polri.go.id/blog/none-22?search=timnas+indonesia'
res_first = requests.get(url_first, headers=headers)
soup_first = BeautifulSoup(res_first.text, 'lxml')

page_numbers = []
for a in soup_first.find_all('a'):
    try:
        num = int(a.text.strip())
        page_numbers.append(num)
    except:
        continue

last_page = max(page_numbers)
print(f"🔎 Total halaman ditemukan {last_page}")

with open('tribata.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Link', 'Time', 'Content'])

    for page in range(1, last_page + 1):
        print(f"\n📄 Scraping halaman {page}...")
        url = f'https://tribratanews.polri.go.id/blog/none-22/page/{page}/?search=timnas+indonesia'
        try:
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            articles = soup.find_all('article')

            for art in articles:
                try:
                    # Menambahkan debug untuk memeriksa struktur artikel
                    print("Debug: Artikel yang sedang diproses:", art)

                    # Mencari tag a dengan kelas 'text-reset text-decoration-none' dan mencari h2 di dalamnya
                    a_tag = art.find('a', class_='text-reset text-decoration-none')
                    if not a_tag:
                        print(f"⚠️ Tidak ditemukan tag <a> pada artikel: {art}")
                        continue

                    title_tag = a_tag.find('h2', class_='title-blog-list h6')
                    if not title_tag:
                        print(f"⚠️ Tidak ditemukan tag <h2> pada artikel: {art}")
                        continue

                    title = title_tag.text.strip()
                    link = a_tag['href']
                    time_info = art.find('span', class_='text-muted pl-2 pr-1').text.strip()
                    content = get_article_content(link)
                    writer.writerow([title, link, time_info, content])
                    print(f"✅ {title}")
                    time.sleep(1)
                except Exception as e:
                    print(f"Error parsing artikel di halaman {page}: {e}")
                    continue

            time.sleep(1)
        except Exception as e:
            print(f"Gagal akses page {page}: {e}")
            continue


🔎 Total halaman ditemukan 3

📄 Scraping halaman 1...
Debug: Artikel yang sedang diproses: <article class="o_wblog_post position-relative" name="blog_post">
<a class="text-decoration-none d-block mb-2" href="/blog/none-22/hoaks-pssi-tolak-ajakan-fifa-untuk-ke-piala-dunia-menggantikan-iran-89833">
<div class="o_record_cover_container d-flex flex-column h-100 o_colored_level o_cc o_list_cover o_not_editable rounded overflow-hidden shadow mb-3" data-name="Cover">
<div class="o_record_cover_component o_record_cover_image" style="background-image: url('/web/image/blog.post/89833/image');"></div>
</div>
</a>
<div>
<a class="text-reset text-decoration-none" href="/blog/none-22/hoaks-pssi-tolak-ajakan-fifa-untuk-ke-piala-dunia-menggantikan-iran-89833">
<h2 class="title-blog-list h6">[HOAKS] PSSI Tolak Ajakan FIFA untuk ke Piala Dunia Menggantikan Iran</h2>
</a>
</div>
<div class="o_wblog_normalize_font pr-2 pb-2">
<div class="align-items-center mt-2 mx-n2 flex-grow-1">
<span class="text-muted p